# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import io
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_data = pd.read_csv("C:/Users/kwelter/Desktop/Repositories/python_APIs/output_data/cities.csv") 
city_weather_data

,Unnamed: 0,City,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,Vaini,-21.20,-175.20,62.60,93,20,4.70,TO,1595413645
1,1,Tuatapere,-46.13,167.68,42.13,80,77,16.89,NZ,1595413645
2,2,Rikitea,-23.12,-134.97,66.58,64,52,16.26,PF,1595413645
3,3,Tiksi,71.69,128.87,50.49,85,72,4.85,RU,1595413519
4,4,Jamestown,42.10,-79.24,68.00,93,40,5.82,US,1595413506
...,...,...,...,...,...,...,...,...,...,...
542,542,Goundam,16.41,-3.67,91.81,38,89,10.80,ML,1595413637
543,543,Muli,2.92,73.57,84.43,73,58,14.23,MV,1595413732
544,544,‘Ewa Beach,21.32,-158.01,78.80,88,20,3.36,US,1595413732
545,545,Port Hedland,-20.32,118.57,71.60,53,0,4.70,AU,1595413582


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Lat and Lon as Locations
locations = city_weather_data[['Lat', 'Lon']].astype(float)
#Humidity as Weight
humidity = city_weather_data['Humidity'].astype(float)

In [5]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [6]:
#center = city_weather_data['Lat'] == 50.00
#fig = gmaps.figure(zoom_level=10, center=center,)
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                dissipating=False, max_intensity=100,
                                 point_radius = 1)


fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [54]:
#Maximum temperature lower than 85 degrees but higher than 70.
max_temp_df = city_weather_data.loc[(city_weather_data['Max_Temp'] >70) &(city_weather_data['Max_Temp'] <85)]

#Wind speed less than 5 mph.
max_temp_wind_df = max_temp_df.loc[(city_weather_data['Wind_Speed'] <5)]

#Cloudiness less than 1
max_temp_wind_noclouds_df = max_temp_wind_df.loc[(city_weather_data['Cloudiness'] <1)]

#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
max_temp_wind_noclouds_df

,Unnamed: 0,City,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
17,17,Cururupu,-1.83,-44.87,75.07,92,0,2.73,BR,1595413648
293,293,Sovetskiy,61.36,63.58,83.03,28,0,3.24,RU,1595413690
302,302,Novozaimka,56.49,66.92,75.81,45,0,4.76,RU,1595413692
368,368,Mossamedes,-15.20,12.15,75.20,64,0,2.01,AO,1595413648
439,439,La Santisima Trinidad,-14.83,-64.90,71.01,66,0,4.63,BO,1595413713
450,450,Piet Retief,-27.01,30.81,71.04,18,0,3.40,ZA,1595413715
488,488,Durban,-29.86,31.03,73.40,40,0,4.70,ZA,1595413512
494,494,Miandrivazo,-19.52,45.47,83.64,23,0,2.91,MG,1595413723
545,545,Port Hedland,-20.32,118.57,71.60,53,0,4.70,AU,1595413582


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
max_temp_wind_noclouds_df["Hotel Name"] = ""
hotel_df = max_temp_wind_noclouds_df


C:\Users\kwelter\Anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
params = {
    "type": "hotel",
    "radius": 5000,
    "key": g_key
}

for index, row in max_temp_wind_noclouds_df.iterrows():
    lat = row["Lat"]
    lng = row["Lon"]
    
    params["location"] = f"{lat}, {lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_data = requests.get(base_url, params=params)
    

    hotel_data = hotel_data.json()
    #print(json.dumps(hotel_data, indent= 4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing")

C:\Users\kwelter\Anaconda3\envs\NewPythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [57]:
# Save Data to csv
hotel_df.to_csv("cities.csv")

# Visualize to confirm airport data appears
hotel_df.head()

,Unnamed: 0,City,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
17,17,Cururupu,-1.83,-44.87,75.07,92,0,2.73,BR,1595413648,Cururupu
293,293,Sovetskiy,61.36,63.58,83.03,28,0,3.24,RU,1595413690,Sovetsky
302,302,Novozaimka,56.49,66.92,75.81,45,0,4.76,RU,1595413692,Novaya Zaimka
368,368,Mossamedes,-15.20,12.15,75.20,64,0,2.01,AO,1595413648,Namibe
439,439,La Santisima Trinidad,-14.83,-64.90,71.01,66,0,4.63,BO,1595413713,Trinidad


In [60]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [61]:
hotel_df

,Unnamed: 0,City,Lat,Lon,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel Name
17,17,Cururupu,-1.83,-44.87,75.07,92,0,2.73,BR,1595413648,Cururupu
293,293,Sovetskiy,61.36,63.58,83.03,28,0,3.24,RU,1595413690,Sovetsky
302,302,Novozaimka,56.49,66.92,75.81,45,0,4.76,RU,1595413692,Novaya Zaimka
368,368,Mossamedes,-15.20,12.15,75.20,64,0,2.01,AO,1595413648,Namibe
439,439,La Santisima Trinidad,-14.83,-64.90,71.01,66,0,4.63,BO,1595413713,Trinidad
450,450,Piet Retief,-27.01,30.81,71.04,18,0,3.40,ZA,1595413715,Piet Retief
488,488,Durban,-29.86,31.03,73.40,40,0,4.70,ZA,1595413512,Durban
494,494,Miandrivazo,-19.52,45.47,83.64,23,0,2.91,MG,1595413723,Miandrivazo
545,545,Port Hedland,-20.32,118.57,71.60,53,0,4.70,AU,1595413582,Port Hedland


In [66]:

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)


# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Display figure
#fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…